In [16]:
import pandas as pd 

pd.set_option("display.max_rows", None)
df = pd.read_csv("res.csv")
df['indexsize'] = round(df['indexsize'] / 1e6, 2)
#df['build'] = round(df['build'] / (60 * 60), 2)


In [9]:
df[['algorithm','dataset', 'indexsize']].groupby(['algorithm', 'dataset']).max()

indexsize
algorithm        dataset                         
BallTree(nmslib) gist-960-euclidean          7.96
                 glove-100-angular           1.37
                 nytimes-256-angular         0.82
                 random-10nn-euclidean       1.98
                 sift-128-euclidean          1.39
NGT-onng         gist-960-euclidean          5.60
                 glove-100-angular           2.31
                 nytimes-256-angular         1.16
                 random-10nn-euclidean       2.96
                 sift-128-euclidean          1.85
NGT-panng        sift-256-hamming            1.99
                 word2bits-800-hamming       1.73
SW-graph(nmslib) gist-960-euclidean          4.11
                 glove-100-angular           1.54
                 nytimes-256-angular         0.56
                 random-10nn-euclidean       1.55
                 sift-128-euclidean          1.27
annoy            gist-960-euclidean          5.24
                 glove-100-angular           7.63
                 nytimes-256-angular         1.54
                 random-10nn-euclidean       6.83
                 sift-128-euclidean          5.77
                 sift-256-hamming            6.61
                 word2bits-800-hamming       2.15
annoy-euclidean  sift-256-hamming            6.70
                 word2bits-800-hamming       3.31
bruteforce-blas  glove-100-angular           0.00
faiss-gpu-bf     sift-128-euclidean          0.63
faiss-ivf        gist-960-euclidean          5.00
                 glove-100-angular           0.66
                 nytimes-256-angular         0.41
                 random-10nn-euclidean       1.22
                 sift-128-euclidean          0.72
                 sift-256-hamming            1.40
                 word2bits-800-hamming       1.68
faiss-ivf-gpu    sift-128-euclidean          0.69
flann            gist-960-euclidean          5.20
                 glove-100-angular           0.78
                 nytimes-256-angular         0.40
                 random-10nn-euclidean       2.03
                 sift-128-euclidean          1.02
hnsw(faiss)      gist-960-euclidean          4.15
                 glove-100-angular           1.39
                 nytimes-256-angular         0.47
                 random-10nn-euclidean       1.09
                 sift-128-euclidean          1.28
hnsw(nmslib)     gist-960-euclidean          5.49
                 glove-100-angular           4.64
                 nytimes-256-angular         0.83
                 random-10nn-euclidean       3.26
                 sift-128-euclidean          2.79
kgraph           gist-960-euclidean          5.77
                 glove-100-angular          13.22
                 nytimes-256-angular         3.80
                 random-10nn-euclidean       2.13
                 sift-128-euclidean          2.41
mih              sift-256-hamming            1.47
                 word2bits-800-hamming       6.54
mrpt             gist-960-euclidean          7.82
pynndescent      gist-960-euclidean          6.69
                 glove-100-angular           6.24
                 nytimes-256-angular         1.01
                 random-10nn-euclidean       6.31
                 sift-128-euclidean          5.13
                 sift-256-hamming            6.14
                 word2bits-800-hamming       3.98
rpforest         glove-100-angular          19.90
                 nytimes-256-angular         5.37

In [22]:
df['search_time'] = 10000 / df['qps']
search_df = df[["algorithm", "dataset", "search_time"]].groupby(["algorithm", "dataset"]).sum().reset_index()
search_df['search_time'] = round(5 * df['search_time'], 2)
search_df

,algorithm,dataset,search_time
0,BallTree(nmslib),gist-960-euclidean,62.38
1,BallTree(nmslib),glove-100-angular,711.34
2,BallTree(nmslib),nytimes-256-angular,105.04
3,BallTree(nmslib),random-10nn-euclidean,26.48
4,BallTree(nmslib),sift-128-euclidean,79.23
5,NGT-onng,gist-960-euclidean,183.56
6,NGT-onng,glove-100-angular,749.77
7,NGT-onng,nytimes-256-angular,79.87
8,NGT-onng,random-10nn-euclidean,618.90
9,NGT-onng,sift-128-euclidean,399.50


In [20]:
searcdh_df = df[["algorithm", "dataset", "qps"]].groupby(["algorithm", "dataset"]).sum().reset_index()
search_df['qps'] = round(5 * 10000 / search_df['qps'] , 2)
search_df

,algorithm,dataset,qps
0,BallTree(nmslib),gist-960-euclidean,29.43
1,BallTree(nmslib),glove-100-angular,8.73
2,BallTree(nmslib),nytimes-256-angular,29.42
3,BallTree(nmslib),random-10nn-euclidean,44.36
4,BallTree(nmslib),sift-128-euclidean,1.18
5,NGT-onng,gist-960-euclidean,1.15
6,NGT-onng,glove-100-angular,0.47
7,NGT-onng,nytimes-256-angular,0.91
8,NGT-onng,random-10nn-euclidean,0.59
9,NGT-onng,sift-128-euclidean,0.34


In [25]:
build_df['total'] = build_df['build'] + search_df['search_time'] / (60 * 60)
build_df.sort_values(by="total")

,algorithm,dataset,build,total
27,faiss-gpu-bf,sift-128-euclidean,0.00,0.024192
23,annoy,word2bits-800-hamming,0.11,0.116000
38,flann,nytimes-256-angular,0.14,0.143361
35,faiss-ivf-gpu,sift-128-euclidean,0.10,0.218192
30,faiss-ivf,nytimes-256-angular,0.23,0.237925
22,annoy,sift-256-hamming,0.25,0.263486
57,mih,word2bits-800-hamming,0.34,0.340672
14,SW-graph(nmslib),nytimes-256-angular,0.37,0.400897
10,NGT-panng,sift-256-hamming,0.38,0.420739
56,mih,sift-256-hamming,0.42,0.420753
